# LLM Optimization Modelling Experiment

In [1]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [160]:
problem = '''You are a city planner, looking to open facilities at some locations. We have a set of customers and a set of possible locations for opening facilities. Each potential location for establishing a facility incurs a fixed annual activation cost, which must be paid if the facility is used, regardless of the service volume it handles. Furthermore, this service volume at each facility is capped at a maximum annual limit. Additionally, there are transportation costs associated with servicing each customer from each facility.
The goal is to minimize the overall costs, which include both the fixed activation costs for any opened facilities and the transportation costs for servicing customers. This must be done while making sure that each customer’s demand is met, each facility does not exceed its maximum service volume, and customers can of course only be serviced by a facility that is opened. 
Please formulate this as a mathematical optimization model.'''

## 2. Ask for parameters

In [203]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [204]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Sets:

* **I**: Set of potential facility locations.
* **J**: Set of customers. 

## Parameters:

* **f<sub>i</sub>**: Fixed annual activation cost of facility *i* ∈ I.
* **C<sub>i</sub>**: Maximum annual service volume of facility *i* ∈ I.
* **d<sub>j</sub>**: Annual demand of customer *j* ∈ J.
* **t<sub>ij</sub>**: Transportation cost of servicing customer *j* ∈ J from facility *i* ∈ I.

## Decision Variables:

* **x<sub>ij</sub>**: Amount of demand of customer *j* ∈ J served by facility *i* ∈ I.
* **y<sub>i</sub>**: Binary variable equal to 1 if facility *i* ∈ I is opened, and 0 otherwise. 


# 2. Ask for objective

In [205]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [206]:
Markdown(response2.text)

Minimize: 
 $$\sum_{i \in I} f_i  y_i + \sum_{i \in I} \sum_{j \in J} t_{ij} x_{ij}$$ 


# 3. Ask for constraints

In [207]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [208]:
Markdown(response3.text)

Subject to:

* **Demand fulfillment:** $\sum_{i \in I} x_{ij} = d_j$ for all $j \in J$ 
* **Capacity constraints:** $\sum_{j \in J} x_{ij} \le C_i y_i$ for all $i \in I$
* **Service only from opened facilities:** $x_{ij} \le d_j y_i$ for all $i \in I$, $j \in J$
* **Non-negative service:**  $x_{ij} \ge 0$ for all $i \in I$, $j \in J$
* **Binary facility opening decisions:** $y_i \in \{0, 1\}$ for all $i \in I$ 


## 3. Generate the pyomo code

In [209]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [210]:
#Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo

# Sample data (replace with your actual data)
facilities = ['Fac1', 'Fac2', 'Fac3']
customers = ['Cust1', 'Cust2', 'Cust3', 'Cust4']

fixed_costs = {'Fac1': 100, 'Fac2': 150, 'Fac3': 120}
capacities = {'Fac1': 200, 'Fac2': 250, 'Fac3': 180}
demands = {'Cust1': 80, 'Cust2': 120, 'Cust3': 70, 'Cust4': 60}
transportation_costs = {
    ('Fac1', 'Cust1'): 5, ('Fac1', 'Cust2'): 8, ('Fac1', 'Cust3'): 6, ('Fac1', 'Cust4'): 4,
    ('Fac2', 'Cust1'): 7, ('Fac2', 'Cust2'): 3, ('Fac2', 'Cust3'): 9, ('Fac2', 'Cust4'): 6,
    ('Fac3', 'Cust1'): 4, ('Fac3', 'Cust2'): 6, ('Fac3', 'Cust3'): 2, ('Fac3', 'Cust4'): 8,
}

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=facilities)  
model.J = pyo.Set(initialize=customers) 

# Parameters
model.f = pyo.Param(model.I, initialize=fixed_costs)
model.C = pyo.Param(model.I, initialize=capacities)
model.d = pyo.Param(model.J, initialize=demands)
model.t = pyo.Param(model.I, model.J, initialize=transportation_costs)

# Decision Variables
model.x = pyo.Var(model.I, model.J, within=pyo.NonNegativeReals)
model.y = pyo.Var(model.I, within=pyo.Binary)

# Objective Function
def objective_rule(model):
    return sum(model.f[i] * model.y[i] for i in model.I) + sum(model.t[i, j] * model.x[i, j] for i in model.I for j in model.J)
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
def demand_fulfillment(model, j):
    return sum(model.x[i, j] for i in model.I) == model.d[j]
model.demand_constraint = pyo.Constraint(model.J, rule=demand_fulfillment)

def capacity_constraint(model, i):
    return sum(model.x[i, j] for j in model.J) <= model.C[i] * model.y[i]
model.capacity_constraint = pyo.Constraint(model.I, rule=capacity_constraint)

def service_from_opened_facilities(model, i, j):
    return model.x[i, j] <= model.d[j] * model.y[i]
model.service_constraint = pyo.Constraint(model.I, model.J, rule=service_from_opened_facilities)

# Solve the model
solver = pyo.SolverFactory('glpk')  # Choose a suitable solver
solver.solve(model)

# Print results
print("Optimal Solution:")
print("Total Cost:", pyo.value(model.obj))
print("Facility Opening Decisions:")
for i in model.I:
    print(f"\t{i}: {pyo.value(model.y[i])}")
print("Service Allocation:")
for i in model.I:
    for j in model.J:
        if pyo.value(model.x[i, j]) > 0:
            print(f"\t{i} serves {j} with amount: {pyo.value(model.x[i, j])}")
```

This code first defines sample data for the problem. **Remember to replace this sample data with your actual data.** The rest of the code sets up the mathematical model using Pyomo, solves it, and prints the results. 


## 4. Input problem data and try running the generated code

In [213]:
import pyomo.environ as pyo

# Sample data (replace with your actual data)
facilities = [1, 2, 3]
customers = [1,2,3,4,5]

fixed_costs = {1: 1000, 2: 1000, 3: 1000}
capacities = {1: 500, 2: 500, 3: 500}  
demands ={1: 80, 2: 270, 3: 250,4:160,5:180}
transportation_costs = {
    (1, 1): 4,  (2, 1): 6,  (3, 1): 9,
    (1, 2): 5,  (2, 2): 4,  (3, 2): 7,
    (1, 3): 6,  (2, 3): 3,  (3, 3): 4,
    (1, 4): 8,  (2, 4): 5,  (3, 4): 3,
    (1, 5): 10, (2, 5): 8,  (3, 5): 4,
}

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=facilities)  
model.J = pyo.Set(initialize=customers) 

# Parameters
model.f = pyo.Param(model.I, initialize=fixed_costs)
model.C = pyo.Param(model.I, initialize=capacities)
model.d = pyo.Param(model.J, initialize=demands)
model.t = pyo.Param(model.I, model.J, initialize=transportation_costs)

# Decision Variables
model.x = pyo.Var(model.I, model.J, within=pyo.NonNegativeReals)
model.y = pyo.Var(model.I, within=pyo.Binary)

# Objective Function
def objective_rule(model):
    return sum(model.f[i] * model.y[i] for i in model.I) + sum(model.t[i, j] * model.x[i, j] for i in model.I for j in model.J)
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
def demand_fulfillment(model, j):
    return sum(model.x[i, j] for i in model.I) == model.d[j]
model.demand_constraint = pyo.Constraint(model.J, rule=demand_fulfillment)

def capacity_constraint(model, i):
    return sum(model.x[i, j] for j in model.J) <= model.C[i] * model.y[i]
model.capacity_constraint = pyo.Constraint(model.I, rule=capacity_constraint)

def service_from_opened_facilities(model, i, j):
    return model.x[i, j] <= model.d[j] * model.y[i]
model.service_constraint = pyo.Constraint(model.I, model.J, rule=service_from_opened_facilities)

# Solve the model
solver = pyo.SolverFactory('glpk')  # Choose a suitable solver
solver.solve(model)

# Print results
print("Optimal Solution:")
print("Total Cost:", pyo.value(model.obj))
print("Facility Opening Decisions:")
for i in model.I:
    print(f"\t{i}: {pyo.value(model.y[i])}")
print("Service Allocation:")
for i in model.I:
    for j in model.J:
        if pyo.value(model.x[i, j]) > 0:
            print(f"\t{i} serves {j} with amount: {pyo.value(model.x[i, j])}")

Optimal Solution:
Total Cost: 5610.0
Facility Opening Decisions:
	1: 0.0
	2: 1.0
	3: 1.0
Service Allocation:
	2 serves 1 with amount: 80.0
	2 serves 2 with amount: 270.0
	2 serves 3 with amount: 150.0
	3 serves 3 with amount: 100.0
	3 serves 4 with amount: 160.0
	3 serves 5 with amount: 180.0


## 5. Correct the code to verify model viability (optional)

## 6. Print the responses

In [214]:
print(response.text)

## Sets:

* **I**: Set of potential facility locations.
* **J**: Set of customers. 

## Parameters:

* **f<sub>i</sub>**: Fixed annual activation cost of facility *i* ∈ I.
* **C<sub>i</sub>**: Maximum annual service volume of facility *i* ∈ I.
* **d<sub>j</sub>**: Annual demand of customer *j* ∈ J.
* **t<sub>ij</sub>**: Transportation cost of servicing customer *j* ∈ J from facility *i* ∈ I.

## Decision Variables:

* **x<sub>ij</sub>**: Amount of demand of customer *j* ∈ J served by facility *i* ∈ I.
* **y<sub>i</sub>**: Binary variable equal to 1 if facility *i* ∈ I is opened, and 0 otherwise. 



In [215]:
print(response2.text)

Minimize: 
 $$\sum_{i \in I} f_i  y_i + \sum_{i \in I} \sum_{j \in J} t_{ij} x_{ij}$$ 



In [216]:
print(response3.text)

Subject to:

* **Demand fulfillment:** $\sum_{i \in I} x_{ij} = d_j$ for all $j \in J$ 
* **Capacity constraints:** $\sum_{j \in J} x_{ij} \le C_i y_i$ for all $i \in I$
* **Service only from opened facilities:** $x_{ij} \le d_j y_i$ for all $i \in I$, $j \in J$
* **Non-negative service:**  $x_{ij} \ge 0$ for all $i \in I$, $j \in J$
* **Binary facility opening decisions:** $y_i \in \{0, 1\}$ for all $i \in I$ 



In [217]:
print(response4.text)

```python
import pyomo.environ as pyo

# Sample data (replace with your actual data)
facilities = ['Fac1', 'Fac2', 'Fac3']
customers = ['Cust1', 'Cust2', 'Cust3', 'Cust4']

fixed_costs = {'Fac1': 100, 'Fac2': 150, 'Fac3': 120}
capacities = {'Fac1': 200, 'Fac2': 250, 'Fac3': 180}
demands = {'Cust1': 80, 'Cust2': 120, 'Cust3': 70, 'Cust4': 60}
transportation_costs = {
    ('Fac1', 'Cust1'): 5, ('Fac1', 'Cust2'): 8, ('Fac1', 'Cust3'): 6, ('Fac1', 'Cust4'): 4,
    ('Fac2', 'Cust1'): 7, ('Fac2', 'Cust2'): 3, ('Fac2', 'Cust3'): 9, ('Fac2', 'Cust4'): 6,
    ('Fac3', 'Cust1'): 4, ('Fac3', 'Cust2'): 6, ('Fac3', 'Cust3'): 2, ('Fac3', 'Cust4'): 8,
}

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=facilities)  
model.J = pyo.Set(initialize=customers) 

# Parameters
model.f = pyo.Param(model.I, initialize=fixed_costs)
model.C = pyo.Param(model.I, initialize=capacities)
model.d = pyo.Param(model.J, initialize=demands)
model.t = pyo.Param(model.I, model.J, initialize=transpo